# 🏥 Affinage (*fine-tuning*) de modèles encodeurs PARTAGES (BERT-like) pour la classification de textes

### Installation des libraries

In [ ]:
# Parfois il est plus simple d'installer torch via conda (ou d'utiliser le
# paquet pytorch). S'il y a des erreurs avec numpy installez 'numpy<2'.

# On utilise `datasets<4` car la version 4 de tokenizer ne permet plus de charger
# des jeux de données à l'aide de scripts de chargement. Certains jeux de
# données utilisent toujours des scripts de chargements (notamment DrBenchmark).

%pip install 'transformers[torch]' 'datasets<4' tensorboardX torch
%pip install scikit-learn numpy

In [ ]:
######################################
###########    A Modifier    #########
######################################

# Si le jeu de donnée, modèle ou tokeniseur n'est pas public sur huggingface il
# faut utiliser un jeton de connexion à créer ici : https://huggingface.co/settings/tokens

# Pour l'instant les modèles PARTAGES sont privés et accessibles avec le jeton
#  communiqué avec ce fichier
HF_TOKEN = ""

# Ici on utilise un des modèles du projet PARTAGES
# A modifier pour utiliser un autre modèle comme base d'affinage
BASE_MODEL_NAME = "PARTAGES-dev/PARTAGES-encoder-v1"

######################################

# Petit hack pour éviter un avertissement spécifique à Google Colab sur le
# chargement du jeton huggingface
try:
  from huggingface_hub.utils import _auth
  _auth._get_token_from_google_colab = lambda: None
except ImportError:
  pass

### Chargement du tokeniseur

In [ ]:
import os
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, token=HF_TOKEN)

### Chargement et préparation des données

Chargement des données, la colonne 'specialities_one_hot' est une liste qui représente les étiquettes a associer à ce document.
Par exemple si l'ensemble des étiquettes possibles est : `immunitaire`, `blessures`, `chimiques` et `virales`. Chaque document sera représenté par une liste de 4 éléments correspondant aux 4 étiquettes. Ainsi un document devant être classé comme `immunitaire` et `virales` aura le vecteur suivant: `[1, 0, 0, 1]`.

```python
[{'id': str, 'text': list(str), 'specialities_one_hot': list(int)}, ...]
```

In [ ]:
def preprocess_function(e, texts_column, labels_column, max_length=None):
    if max_length is None:
        max_length = tokenizer.model_max_length
    res = tokenizer(e[texts_column], truncation=True, max_length=max_length, padding='do_not_pad')
    # Pas de padding lors du prétraitement, c'est le DataCollator qui s'en chargera plus tard.
    res["label"] = e[labels_column]
    return res

In [ ]:
import torch
def one_hot_encoding(labels, num_labels):
    vector = torch.zeros(num_labels)
    vector[labels] = 1
    return vector

In [ ]:
#########################################################
###########    ZONE A Modifier POUR UTILISER    #########
###########         VOS PROPRES DONNEES         #########
##          ATTENTION AUX DONNEES PRIVEES NE PAS       ##
##              EXECUTER SUR LE CLOUD                  ##
#########################################################
# En sortie de ces cellule:
#  - label_list : liste des différentes étiquettes
#  - train_dataset : un Dataset tokenisé avec une colonne "label"
#  - eval_dataset : un Dataset tokenisé avec une colonne "label"
#  - PROBLEM_TYPE : qui sera utilisé pour charger le modèle 

# L'exemple est avec le corpus DEFT2021 que l'on peut obtenir via la collection
#  de DrBenchmark (cf github.com/DrBenchmark/DrBenchmark)
# https://talnarchives.atala.org/ateliers/2021/DEFT/77.pdf
# Chaque cas clinique est associé à des etiquettes décrivant le profil cliniques
#  du patient. Après extraction, le corps du texte est tokenisé via la fonction
#  preprocess_function.

import datasets

dataset = datasets.load_dataset('DrBenchmark/DEFT2021', 'cls', trust_remote_code=True)
dataset

In [ ]:
label_column = "specialities"

print("Exemple d'étiquettes:")
for doc in dataset['train'].select(range(3))[label_column]:
    print(doc)

In [ ]:
# Si plusieurs etiquettes par document
PROBLEM_TYPE = "multi_label_classification"

label_list = dataset["train"].features[label_column].feature.names

# Si Attribute Error: les features du dataset ne sont pas définies, on récupère
#  toutes les étiquettes de tout les documents
#label_list = list({label for split in dataset for doc in dataset[split][label_column] for label in doc})

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in id2label.items()}
print("Dictionnaire étiquette->identifiant:")
print(id2label)

# On copie les labels dans la colonne 'label' pour en faciliter la modification
dataset = dataset.map(lambda d: {'label': d[label_column]})

# Si les etiquettes sont des chaines de caractère il faut les transformer en entier
# dataset = dataset.map(lambda d: {'label': [label2id[l] for l in d['label']]})
# On encode les etiquettes en vecteur
dataset = dataset.map(lambda d: {'label': one_hot_encoding(d['label'], len(id2label))})

# On s'assure que les labels sont bien des flottants (pour le calcul de la loss)
dataset = dataset.cast_column('label', datasets.Sequence(datasets.Value("float")))

# Affichage d'un exemple pour vérifier que tout est correct
print(dataset['train'][0])

In [ ]:
# Si une etiquette par document
"""
PROBLEM_TYPE = "single_label_classification"

label_list = dataset["train"].features[label_column].feature.names

# Si Attribute Error: les features du dataset ne sont pas définies, on récupère
#  toutes les étiquettes de tout les documents
#label_list = list({label for split in dataset for label in dataset[split][label_column]})

id2label = {i: l for i, l in enumerate(label_list)}
label2id = {l: i for i, l in id2label.items()}
print(id2label)

# On copie les labels dans la colonne 'label' pour en faciliter la modification
dataset = dataset.map(lambda d: {'label': d[label_column]})

# Si les etiquettes sont des chaines de caractère il faut les transformer en entier
#dataset = dataset.map(lambda d: {'label': label2id[d['label']]})
"""

In [1]:
# Tokenize dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    fn_kwargs={
        'texts_column': 'text',
        'labels_column': 'label',
        # On met un maximum a 8192 pour éviter les erreurs de mémoire liés au
        #  traitement de long documents mais c'est a modifier selon les besoins
        #  et capacités de calcul. Certains modèle n'ayant pas de limite.
        'max_length': min(tokenizer.model_max_length, 8192),
    },
    batched=True, batch_size=100,
)

# Split dataset into train and validation sets
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

#############################################

NameError: name 'dataset' is not defined

In [ ]:
# Affichage de quelques statistiques du corpus
import random
import textwrap

import numpy as np

# Affichage d'exemples au hasard
to_show = [15, random.randrange(len(train_dataset))]

print(f"Etiquettes : {label_list}")
idx_train_longest = int(np.argmax([len(e) for e in train_dataset['input_ids']]))
print(f"Plus long  doc train : idx {idx_train_longest:4d}, len {len(train_dataset[idx_train_longest]['input_ids']):4d}")
idx_train_shortest = int(np.argmin([len(e) for e in train_dataset['input_ids']]))
print(f"Plus court doc train : idx {idx_train_shortest:4d}, len {len(train_dataset[idx_train_shortest]['input_ids']):4d}")

idx_eval_longest = int(np.argmax([len(e) for e in eval_dataset['input_ids']]))
print(f"Plus long  doc valid : idx {idx_eval_longest:4d}, len {len(eval_dataset[idx_eval_longest]['input_ids']):4d}")
idx_eval_shortest = int(np.argmin([len(e) for e in eval_dataset['input_ids']]))
print(f"Plus court doc valid : idx {idx_eval_shortest:4d}, len {len(eval_dataset[idx_eval_shortest]['input_ids']):4d}")

for i in to_show:
    doc = train_dataset[i]
    print(f"------ Document train {i} ------")
    print(f"Nb. tokens: {len(doc['input_ids'])}")
    tokens = ' '.join(tokenizer.convert_ids_to_tokens(doc['input_ids'])).replace('\n', '\\n')
    print(f"Tokens: {'\n'.join(textwrap.wrap(tokens, width=120, break_long_words=False, break_on_hyphens=False))}")
    print(f"Etiquettes: {doc['label']}")

### Définition des métriques a rapporter

In [ ]:
# Pour calculer le f1-score pendant l'entrainement
CLASSIFICATION_THRESHOLD = 0.70
# On associe l'étiquette au document si sa probabilité est supérieure au seuil
# (utilisé dans `compute_metrics`)

In [ ]:
#######################################
#### A modifier selon les besoins #####
#######################################

import torch
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score


def toLogits(predictions, threshold):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))

    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    return y_pred


def multi_label_metrics(predictions, labels, threshold):
    y_pred = toLogits(predictions, threshold)
    y_true = labels

    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=.0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=.0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=.0)
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')

    metrics = {'f1_macro': f1_macro_average, 'f1_micro': f1_micro_average, 'f1_weighted': f1_weighted_average, 'roc': roc_auc}

    return metrics


def single_label_metrics(predictions, labels):
    y_pred = np.argmax(predictions, axis=1)
    y_true = labels

    average = "binary" if predictions.shape[-1] == 2 else "weighted"
    
    f1 = f1_score(y_true=y_true, y_pred=y_pred, zero_division=.0, average=average)
    accuracy = accuracy_score(y_true, y_pred)

    metrics = {'f1': f1, 'accuracy': accuracy}

    return metrics

In [ ]:
def compute_metrics(p):
    global CLASSIFICATION_THRESHOLD, PROBLEM_TYPE
    preds = p.predictions
    if isinstance(preds, tuple):
        preds = preds[0]

    if PROBLEM_TYPE == "multi_label_classification":
        result = multi_label_metrics(
            predictions=preds,
            labels=p.label_ids,
            threshold=CLASSIFICATION_THRESHOLD
        )
    else:
        result = single_label_metrics(
            predictions=preds,
            labels=p.label_ids,
        )
    return result

### Chargement du modèle de base


Maintenant que l'on a chargé les données on connait le nombre d'étiquettes donc
on peut charger le modèle et initialiser sa couche de classification.

C'est ici que `PROBLEM_TYPE` entre en jeu:
- `single_label_classification` : une seule étiquette par document
- `multi_label_classification` : plusieurs étiquettes par document

In [ ]:
#############################################
############    A Modifier    ###############
#############################################

SEED = 42  # Pour faciliter la reproductibilité des résultats

#############################################

In [ ]:
from transformers import set_seed
from transformers import AutoModelForSequenceClassification

# On initialise le générateur de nombres aléatoires avant de charger le modèle
# pour s'assurer que l'initialisation des poids de la couche de classification
# soit toujours la même.
set_seed(SEED)
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_NAME, token=HF_TOKEN, num_labels=len(label_list),
    problem_type=PROBLEM_TYPE,
    # Pour avoir de belles etiquettes en sortie du modèle
    id2label = {i: l for i, l in enumerate(label_list)},
    label2id = {l: i for i, l in enumerate(label_list)},
)
print(model)

### Préparation de l'entrainement

In [ ]:
#############################################
############    A Modifier    ###############
#############################################

# Nom du modèle qui va être entrainé
OUTPUT_MODEL_NAME = "PARTAGES-encoder-v1-classif"
# Chemin où il va être sauvegarder
OUTPUT_DIR = os.path.join(os.environ['HOME'], "PARTAGES", OUTPUT_MODEL_NAME)

#############################################

Le résultat de l'entrainement sera stocké dans:
```
~/PARTAGES/PARTAGES-encoder-v1-classif/
- checkpoints/
  - checkpoint-10
  - checkpoint-20
  - checkpoint-30
  - checkpoint-...
- training_args.json
- model.pt
- tokenizer.
```

Ainsi les points de contrôles (_checkpoints_) sont conservés, et il sera possible de charger le meilleur modèle résultant de l'entrainement en écrivant:

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_path = os.path.join(os.environ['HOME'], "PARTAGES", "PARTAGES-encoder-v1-classif")
# /!\ Il est important d'utiliser AutoModelForTokenClassification et non
# AutoModel (qui ne va pas charger la couche de classification)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Pour charger un checkpoint spécifique
model = AutoModelForSequenceClassification.from_pretrained(model_path + "/checkpoints/checkpoint-20")
tokenizer = AutoTokenizer.from_pretrained(model_path)
```

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer)

# Pour plus d'arguments
# https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments

training_args = TrainingArguments(
    os.path.join(OUTPUT_DIR, 'checkpoints'),
    overwrite_output_dir=False,  # pour continuer l'entrainement
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2.0e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,
    num_train_epochs=15,
    weight_decay=0.01,
    metric_for_best_model="f1_weighted" if PROBLEM_TYPE == "multi_label_classification" else "f1",
    load_best_model_at_end=True,
    greater_is_better=True,
    logging_strategy="epoch",
    save_only_model=True,
    save_total_limit=5,
    report_to='tensorboard',
)

# Pour augmenter le batch_size mais ne pas avoir d'erreur de mémoire il est possible
#  d'utiliser le paramètre `gradient_accumulation_steps` qui attend N batchs avant
#  d'effectuer la rétropropagation. Par exemple `batch_size=64, gradient_accumulation_steps=1`
#  est équivalent à `batch_size=32, gradient_accumulation_steps=2` ou
#  `batch_size=16, gradient_accumulation_steps=4`
if os.path.exists(OUTPUT_DIR):
    print(f"""Le dossier de sortie existe déjà !
L'entrainement va continuer s'il a été arrêté ou ne rien faire si l'entrainement a terminé.
Chemin : {OUTPUT_DIR}
Soyez sûr de ce qu'il se passe ou bien supprimez le dossier.""")

In [ ]:
# Exemple de document passé par le DataCollator (ajout de padding pour que tout
#  les documents d'un même lot (batch) aient la même longueur
sampled_batch = train_dataset.select_columns(['input_ids']).batch(8)[0]
collated_batch = data_collator(sampled_batch)
collated_doc = collated_batch['input_ids'][0]

print("Exemple de document passé par le Data Collator (ce que le modèle va voir):")
doc_to_print = ' '.join(tokenizer.convert_ids_to_tokens(collated_doc)).replace('\n', '\\n')
print('\n'.join(textwrap.wrap(doc_to_print, width=120, break_long_words=False, break_on_hyphens=False)))

In [ ]:
# Supprimer ou mettre à `False` pour un entrainement réel
debugging_mode = True
if debugging_mode:
    n_samples = min(len(train_dataset), len(eval_dataset))
    n_samples = min(n_samples, 20)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!!            Mode de debug         !!!")
    print(f"!!!  Seuls {n_samples} examples sont utilisés  !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    train_dataset = train_dataset.select(range(n_samples))
    eval_dataset = eval_dataset.select(range(n_samples))

In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    processing_class=tokenizer,
    # Avec EarlyStopping, l'entrainement va s'arrêter après 3 epoch si
    #  la f1_weighted (`metric_for_best_model`) ne s'améliore pas
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics,
)

In [ ]:
# Si erreur de taille vérifiez le paramètre `problem_type` ou `num_labels` au chargement du modèle
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
import json
trainer.save_model(os.path.join(OUTPUT_DIR))
with open(os.path.join(OUTPUT_DIR, "training_args.json"), 'w') as f:
  json.dump(json.loads(training_args.to_json_string()) | {'basemodel_name_or_path': model.name_or_path}, f)

## Utilisation du modèle entrainé

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_path = os.path.join(os.environ['HOME'], "PARTAGES", OUTPUT_MODEL_NAME)
# /!\ Il est important d'utiliser AutoModelForTokenClassification et non
# AutoModel (qui ne va pas charger la couche de classification)

In [ ]:
from transformers import pipeline
print(f"Chargement du modèle {OUTPUT_MODEL_NAME}.")
pipe = pipeline("text-classification", model=model_path, device="cpu")

In [ ]:
def preprocessing(text):
    # Si un pré-traitement a été effectué pour créer le corpus d'entrainement
    #  il faut reproduire ce prétraitement ici.
    return text.replace("c'est", "c' est")

texts = [
    "La patiente est enrhummée, et a pris du paracétamol.",
    "Une deuxième phrase."
]
texts = [preprocessing(t) for t in texts]
predictions = pipe(
    texts,
    batch_size=2,
    top_k=None
)

In [ ]:
# Filtrage des étiquettes qui ont obtenu une probabilitée > a 0.7

# Pour mieux choisir le seuil on peux par exemple:
# - chercher le seuil qui maximise le f1 score sur le corpus de validation
# - chercher un seuil par etiquette qui maximise le f1 score sur le corpus de validation

# Note: l'etiquette sosy signifie "Signes Ou SYmptomes"
for t, p in zip(texts, predictions):
    chosen_labels = [l for l in p if l['score'] > 0.4]
    print(f"Le modèle a assigné les étiquettes suivantes:")
    print(chosen_labels)
    print(f"au document: {t[:50]} [...]")
    print()


## Hyperparameter tuning

Suivant le temps de calcul à votre disposition et de la taille de vos jeux de données. Il est possible de chercher de meilleurs hyperparamètres automatiquement à l'aide de la bibliothèque `optuna`.

In [ ]:
%pip install optuna

On choisit ici de chercher de meilleures valeurs pour la `learning_rate`, le `weight_decay` et une meilleure `seed` qui initialise les paramètres.

In [ ]:
from optuna import Trial

def hp_space(trial: Trial) -> dict[str, float | int | str]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 2e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.005, 0.1, log=True),
        "seed": trial.suggest_int("seed", 1, 40),
    }

# Utilisation d'une fonction model_init pour s'assurer de la reproducibilité
def model_init():
    global label_list, BASE_MODEL_NAME, HF_TOKEN, PROBLEM_TYPE
    return AutoModelForSequenceClassification.from_pretrained(
        BASE_MODEL_NAME, token=HF_TOKEN, num_labels=len(label_list),
        problem_type=PROBLEM_TYPE,
        # Pour avoir de belles etiquettes en sortie du modèle
        id2label = {i: l for i, l in enumerate(label_list)},
        label2id = {l: i for i, l in enumerate(label_list)},
    )

# modification du trainer déjà initialisé avant pour bien ré-initialiser le
#  modèle à chaque essai
trainer.model_init = model_init

In [ ]:
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=3)

In [ ]:
best_run